In [1]:
import sys
import os
import cv2
import numpy as np
import pandas as pd
import shutil
from PyQt5.QtWidgets import QApplication, QMainWindow, QLabel, QPushButton, QVBoxLayout, QWidget, QFileDialog, QScrollArea
from PyQt5.QtCore import Qt
from PyQt5.QtGui import QPixmap, QFont  
from keras.models import load_model

In [2]:
class MusicRecommenderApp(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Music Recommender")
        self.setGeometry(100, 100, 500, 500)  
       
        self.central_widget = QWidget()
        self.setCentralWidget(self.central_widget)
       
        self.upload_button = QPushButton("Upload Image", self)
        self.upload_button.setStyleSheet("background-color: purple; color: white")  
        self.upload_button.setFont(QFont("Arial", 12, QFont.Bold))  
        self.upload_button.clicked.connect(self.upload_image)
       
        self.image_label = QLabel(self)  
        self.image_label.setAlignment(Qt.AlignmentFlag.AlignCenter)  
        self.image_label.setFixedSize(400, 400)  
        self.image_label.setPixmap(QPixmap("default_image.png"))  

        self.result_label = QLabel(self)
        self.result_label.setAlignment(Qt.AlignmentFlag.AlignCenter)
        self.result_label.setFont(QFont("Arial", 14, QFont.Bold))  
        self.result_label.setText("Emotion based Music Recommendation System")  
        self.result_label.setWordWrap(True)  
       
        self.scroll_area = QScrollArea(self)  
        self.scroll_area.setWidgetResizable(True)  
        self.scroll_area.setWidget(self.result_label)  
        self.scroll_area.hide()  
       
        self.layout = QVBoxLayout(self.central_widget)
        self.layout.addWidget(self.upload_button)
        self.layout.addWidget(self.image_label, alignment=Qt.AlignmentFlag.AlignCenter)  
        self.layout.addWidget(self.scroll_area)  
        
        self.model = load_model("C:\\Users\\ekaushik\\Desktop\\Multimodal-MusicRecommender\\Music-Recommender\\best_model.h5")
        self.emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}
       
        self.song_directory = 'C:/Users/ekaushik/Desktop/Multimodal-MusicRecommender/Music-Recommender/songs'
        self.uploaded_images_directory = 'C:/Users/ekaushik/Desktop/Multimodal-MusicRecommender/Music-Recommender/Uploaded Images'
       
        os.makedirs(self.uploaded_images_directory, exist_ok=True)
        
    def detect_emotion(self, image_path):
        image = cv2.imread(image_path)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
       
        for (x,y,w,h) in faces:
            roi_gray = gray[y:y+h, x:x+w]
            roi_gray = cv2.resize(roi_gray, (48, 48))
            roi = roi_gray.astype('float') / 255.0
            roi = np.expand_dims(roi, axis=0)
            roi = np.expand_dims(roi, axis=3)
 
            prediction = self.model.predict(roi)[0]
            predicted_emotion = np.argmax(prediction)
            return self.emotion_dict[predicted_emotion]
       
        return "No face detected"
       
    def recommend_songs(self, emotion):
        csv_path = os.path.join(self.song_directory, f'{emotion.lower()}.csv')
        if os.path.exists(csv_path):
            df = pd.read_csv(csv_path)
            songs = df['Name'].values
            return songs
        else:
            return []
       
    def upload_image(self):
        options = QFileDialog.Options()
        file_path, _ = QFileDialog.getOpenFileName(self, "Upload Image", "", "Image Files (*.png *.jpg *.jpeg)", options=options)
        if file_path:
            emotion = self.detect_emotion(file_path)
            songs = self.recommend_songs(emotion)
            result_text = f"Detected emotion: {emotion}\n\nRecommended songs:\n"
            for song in songs:
                result_text += f"- {song}\n"
            self.result_label.setText(result_text)
            self.scroll_area.show()  
           
          
            pixmap = QPixmap(file_path)
            pixmap = pixmap.scaledToWidth(400)  
            self.image_label.setPixmap(pixmap)
           
         
            file_name = os.path.basename(file_path)
            destination_path = os.path.join(self.uploaded_images_directory, file_name)
            shutil.copy(file_path, destination_path)
            print(f"Uploaded image saved at: {destination_path}")

In [3]:

if __name__ == '__main__':
    app = QApplication(sys.argv)
    window = MusicRecommenderApp()
    window.show()
    sys.exit(app.exec_())

1/1 [==============================] - 0s 150ms/step
Uploaded image saved at: C:/Users/ekaushik/Desktop/Multimodal-MusicRecommender/Music-Recommender/Uploaded Images\Screenshot 2024-04-14 225857.png
1/1 [==============================] - 0s 41ms/step
Uploaded image saved at: C:/Users/ekaushik/Desktop/Multimodal-MusicRecommender/Music-Recommender/Uploaded Images\Screenshot 2024-04-14 230124.png
1/1 [==============================] - 0s 32ms/step
Uploaded image saved at: C:/Users/ekaushik/Desktop/Multimodal-MusicRecommender/Music-Recommender/Uploaded Images\Screenshot 2024-04-14 230238.png
Uploaded image saved at: C:/Users/ekaushik/Desktop/Multimodal-MusicRecommender/Music-Recommender/Uploaded Images\Screenshot 2024-04-14 230321.png
1/1 [==============================] - 0s 27ms/step
Uploaded image saved at: C:/Users/ekaushik/Desktop/Multimodal-MusicRecommender/Music-Recommender/Uploaded Images\WhatsApp Image 2024-03-05 at 2.43.19 PM.jpeg
1/1 [==============================] - 0s 31ms/st

SystemExit: 0

c:\Users\ekaushik\Desktop\Multimodal-MusicRecommender\venv3.11\lib\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
